In [1]:
import pandas
import torch
import matplotlib.pyplot as plt
import seaborn as sns
import ipywidgets as widgets
import json
import re
import math

In [2]:
numerical_cols, object_cols, features = [], [], []
datarfame = pandas.DataFrame()

In [3]:
def data_clean(data):
    data = re.sub('[^A-Za-z0-9.\']+', "", data)
    try:
        num = float(data)
        return num
    except:
        return data

In [4]:
def on_change(change):
    if change['type'] == 'change' and change['name'] == 'value':
        print("changed to %s" % change['new'])

In [5]:
def draw_graph(corr_matrix):
    sns.set()
    if numerical_cols:
        print("heat map: ")
        fig, ax = plt.subplots(figsize=(len(numerical_cols)*1.5, len(numerical_cols)*1.5))
        ax = sns.heatmap(corr_matrix, vmin=-1, vmax=1, center=0, cmap=sns.diverging_palette(20, 220, n=200), square=True)
        #ax.set_xticklabels(ax.get_xticklabels(), rotation=90, horizontalalignment='right')

        print("numerical data distribution: ")
        # fig, axes = plt.subplots(3, 2, sharex=True, figsize=(10,5))
        for itr, col in enumerate(numerical_cols):
            tmp_col = dataframe[col].dropna()
            # plt.subplot(3, 2, itr+1)
            sns.displot(tmp_col, palette="set1", kde=True)
    
        with widget_graph_num:
            display(plt.show())
            
    if object_cols:
        i = 0
        for col in object_cols:
            pic_size, explode = [], []
            labels = dataframe[col].unique()

            if len(labels) < 11:
                plt.figure(i)
                for item in labels:
                    explode.append(0.05)
                    pic_size.append(round((len(dataframe[dataframe[col] == item])/len(dataframe))*100, 2))

                plt.title("Ratio of data in field: " + col)
                plt.pie(pic_size, labels=labels, autopct='%1.1f%%', startangle=90, pctdistance=0.85, explode = explode)
                centre_circle = plt.Circle((0, 0), 0.70, fc='white')
                fig = plt.gcf()
                fig.gca().add_artist(centre_circle)
                plt.tight_layout()
                i += 1
    
        with widget_graph_cat:
            display(plt.show())

In [6]:
def start_process(change):
    global features, dataframe
    file_data = btn_upload.metadata
    file_name = file_data[0]['name']
    file_type = file_name.split(".")[-1]
    
    raw_data = btn_upload.data
    input_list = str(raw_data[0]).split(",")
    
    input_list_cleaned, rows = [], []
    element_length = len(input_list)
    for element in input_list:
        if "\\n" in element:
            element = element.replace("\\r", "")
            pre, post = element.split("\\n")
            rows.append(data_clean(pre))
            input_list_cleaned.append(rows.copy())
            rows.clear()
            rows.append(data_clean(post))
        else:
            rows.append(data_clean(element))
    
    input_list_cleaned[0][0] = str(input_list_cleaned[0][0]).replace("b'", "")
    features = input_list_cleaned.pop(0)
    
    dataframe = pandas.DataFrame(input_list_cleaned, columns=features)
    with widget_data_describe:
        display(dataframe.describe())
    step2()

In [7]:
 def analyze(change):
    target_column_name = dropdown.value
    target_column = dataframe[target_column_name]
    learning_type = "Classification" if dataframe[target_column_name].dtype == "object" else "Classification" if dataframe[target_column_name].sum() == math.ceil(dataframe[target_column_name].sum()) else "Regression"
    with widget_process_type:
        display("Analysed model type: " + learning_type)
    
    corr_matrix = dataframe.corr()
    with widget_corr_matrix:
        display(corr_matrix)
    
    for feature in features:
        numerical_cols.append(feature) if dataframe[feature].dtype == "float64" else object_cols.append(feature)
    
    draw_graph(corr_matrix)

In [8]:
btn_upload = widgets.FileUpload()
btn_run = widgets.Button(description='Process data')
dropdown = widgets.Dropdown(options=['0', '1'], value='0', description='Target label: ')
btn_run2 = widgets.Button(description='Analyze')
btn_run.on_click(start_process)
btn_run2.on_click(analyze)

In [9]:
def step2():
    global dropdown
    dropdown = widgets.Dropdown(options=features, value=features[-1], description='Target label: ')
    dropdown.observe(on_change)
    with widget_dropdown:
        display(dropdown)
    
    with widget_analyze:
        display(btn_run2)

In [10]:
widget_data_describe = widgets.Output()
widget_dropdown = widgets.Output()
widget_process_type = widgets.Output()
widget_corr_matrix = widgets.Output()
widget_process_type = widgets.Output()
widget_analyze = widgets.Output()
widget_graph_num = widgets.Output()
widget_graph_cat = widgets.Output()

widgets.VBox([btn_upload, btn_run, widget_data_describe, widget_dropdown, widget_analyze, widget_process_type, widget_corr_matrix, widget_graph_num, widget_graph_cat])